In [1]:
## アヤメの種類を学習するCNNの実装(2)
## ミニバッチを使用した学習
import numpy as np
from chainer import cuda, Function, gradient_check,\
    Variable, optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

In [2]:
from sklearn import datasets

## アヤメに関する4次元x150個のデータ
iris = datasets.load_iris()
## 入力データ
X = iris.data.astype(np.float32)
## 訓練データ
Y = iris.target
## 入力データサイズ
N = Y.size

print ("shape of iris.data : ", iris.data.shape)

shape of iris.data :  (150, 4)


In [3]:
## アヤメは3種類に分類されるので訓練データ3N次元
Y2 = np.zeros(3*N).reshape(N,3).astype(np.float32)
for i in range(N):
## 正解の種類は1.0, それ以外の2種類は0.0とする
    Y2[i,Y[i]] = 1.0

In [4]:
## 奇数noデータを訓練データ,偶数noデータを検証用に設定
index = np.arange(N)
xtrain =  X[index[index % 2 != 0],:]
ytrain = Y2[index[index % 2 != 0],:]
xtest  =  X[index[index % 2 == 0],:]
yans   =  Y[index[index % 2 == 0]]

In [6]:
## 4x6x3のCNNモデル定義
class IrisChain(Chain):
    def __init__(self):
        super(IrisChain, self).__init__(
            l1 = L.Linear(4,6),
            l2 = L.Linear(6,3),
        )
        
    ## callで誤差関数を定義
    def __call__(self, x, y):
        return F.mean_squared_error(self.fwd(x), y)
    
    ## 順伝搬はcallと別に定義
    def fwd(self, x):
        h1 = F.sigmoid(self.l1(x))
        h2 = self.l2(h1)
        return h2

In [7]:
## CNNモデル初期化と最適化手法設定
model = IrisChain()
optimizer = optimizers.SGD()
optimizer.setup(model)

In [8]:
## ミニバッチを作成して学習開始
n = 74     # 訓練データ数
bs = 25    # バッチサイズ

In [9]:
for j in range(5000):
    ## 入力データをランダムに入れ換えてバッチ化
    sffindx = np.random.permutation(n)
    for i in range(0, n, bs):
        x = Variable(xtrain[sffindx[i:(i+bs) if (i+bs)<n else n]])
        y = Variable(ytrain[sffindx[i:(i+bs) if (i+bs)<n else n]])
        model.zerograds()
        loss = model(x,y)
        loss.backward()
        optimizer.update()

In [10]:
## 検証用データxtで学習結果をテスト
xt = Variable(xtest, volatile='on')
yy = model.fwd(xt)
ans = yy.data
nrow, ncol = ans.shape

ok = 0
for i in range(nrow):
    ## 確率が最大の結果をclsに出力
    cls = np.argmax(ans[i,:])
    print (ans[i,:], cls)
    if cls == yans[i]:
        ok += 1

print ("correct rate : ", ok, "/", nrow, "=", (ok*1.0)/nrow)

[ 1.01045752 -0.0187353  -0.00392096] 0
[ 0.97666955  0.01025237 -0.02512817] 0
[  1.01678991e+00  -2.64389515e-02   8.56831670e-04] 0
[ 0.97331762  0.02611993 -0.02175015] 0
[ 0.90197885  0.1086915  -0.06505185] 0
[ 1.02685308 -0.03031245  0.00748865] 0
[ 0.94491345  0.05865102 -0.0465225 ] 0
[ 1.08462584 -0.11897066  0.04236497] 0
[ 1.05860138 -0.08228382  0.02918707] 0
[ 1.0138644   0.002158    0.00391626] 0
[ 0.96111649  0.07158832 -0.03012744] 0
[ 1.04670787 -0.09766026  0.01790287] 0
[ 0.87371475  0.21247302 -0.07298727] 0
[ 0.94984406  0.07832043 -0.03146571] 0
[ 1.00375772 -0.01071262 -0.00895411] 0
[ 0.90971684  0.12748937 -0.05955537] 0
[ 1.05888844 -0.07071184  0.02845153] 0
[ 0.94355226  0.07243766 -0.04574585] 0
[ 1.03397036 -0.05517808  0.00793619] 0
[ 0.93680722  0.05465029 -0.04700104] 0
[  1.01596284e+00  -3.37509364e-02   6.29022717e-04] 0
[ 0.96204239  0.02558042 -0.03192686] 0
[ 0.93777716  0.12653048 -0.03201178] 0
[  1.01053238e+00  -3.86044383e-04   4.18603420e-0